## Wine classifier with MLFlow, MINIO, and Seldon

In this workshop, we will run an ML Flow model, save the artefacts to MinIO, and create a v2 protocol SeldonDeployment via the SDK

We will follow these steps:
1. Create environment, install libraries, import methods
2. Run experiment and train model in MLFlow
3. Conda pack the environment
4. Push artefacts to MinIO
5. Deploy via the SDK
6. [Optional] Add model metadata via the SDK
7. [Optional] Create a new MLFlow run with different hyperparametes, and deploy a canary model
8. [Optional] Train and deploy a drift detector
9. [Optional] Train and deploy an explainer

### 1. Create environment, install libraries, import methods

In your terminal, create a new virtual environment:
`conda create -n mlflow-wine python=3.8`

Activate the new environment:
`conda activate mlflow-wine`

Check that you are in the correct environment

In [2]:
!conda env list

# conda environments:
#
base                     /Users/josh/opt/anaconda3
ansible-learn            /Users/josh/opt/anaconda3/envs/ansible-learn
beyond-limits            /Users/josh/opt/anaconda3/envs/beyond-limits
core-v2                  /Users/josh/opt/anaconda3/envs/core-v2
da-mlserver              /Users/josh/opt/anaconda3/envs/da-mlserver
data-science             /Users/josh/opt/anaconda3/envs/data-science
deploy-advance           /Users/josh/opt/anaconda3/envs/deploy-advance
deploy-docs              /Users/josh/opt/anaconda3/envs/deploy-docs
deploy-env               /Users/josh/opt/anaconda3/envs/deploy-env
deployboo                /Users/josh/opt/anaconda3/envs/deployboo
firstpipe                /Users/josh/opt/anaconda3/envs/firstpipe
fraud                    /Users/josh/opt/anaconda3/envs/fraud
jwt-test                 /Users/josh/opt/anaconda3/envs/jwt-test
manufactoring-workshop     /Users/josh/opt/anaconda3/envs/manufactoring-workshop
mlflow                   /Users/josh/o

Install required dependencies into your environment

In [3]:
!pip install minio
!pip install conda-pack
!pip install mlserver
!pip install mlserver-mlflow
!pip install sklearn

In [6]:
from minio import Minio
import pandas as pd
import json
import requests
import os
import glob

### 2. Run experiment and train model in MLFlow
For our example, we will use the elastic net wine example from [MLflow's tutorial](https://github.com/mlflow/mlflow/tree/master/examples/sklearn_elasticnet_wine).

Let's load the data to see what's inside:

In [24]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
# Wine Quality Sample
def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [8]:
train(0.5, 0.9)

Elasticnet model (alpha=0.500000, l1_ratio=0.900000):
  RMSE: 0.8323747376136406
  MAE: 0.6669171677143245
  R2: 0.018316455219614114


In [9]:
train(0.2, 0.3)

Elasticnet model (alpha=0.200000, l1_ratio=0.300000):
  RMSE: 0.7357092639331829
  MAE: 0.5667609266233856
  R2: 0.23308686049080007


In [10]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984664
  R2: 0.2799376066653344


Each of these models can actually be found on the `mlruns` folder:

In [9]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.8.0-1).
The following package was automatically installed and is no longer required:
  libfwupdplugin1
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [4]:
!tree -L 1 mlruns/0

mlruns/0  [error opening dir]

0 directories, 0 files


Inside each of these folders, MLflow stores the parameters we used to train our model, any metric we logged during training, and a snapshot of our model. If we look into one of them, we can see the following structure:

In [9]:
!tree mlruns/0/$(ls mlruns/0 | head -1)

mlruns/0/62be42b3506b42aaa2f506f96368a442
├── artifacts
│   └── model
│       ├── conda.yaml
│       ├── MLmodel
│       ├── model.pkl
│       ├── python_env.yaml
│       └── requirements.txt
├── meta.yaml
├── metrics
│   ├── mae
│   ├── r2
│   └── rmse
├── params
│   ├── alpha
│   └── l1_ratio
└── tags
    ├── mlflow.log-model.history
    ├── mlflow.source.name
    ├── mlflow.source.type
    └── mlflow.user

5 directories, 15 files


In particular, we are interested in the MLmodel file stored under artifacts/model:

In [10]:
!cat mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/MLmodel

artifact_path: model
flavors:
  python_function:
    env: conda.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    python_version: 3.8.13
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.1.2
mlflow_version: 1.28.0
model_uuid: b658b250b00d4c1389267e06bc79161a
run_id: 62be42b3506b42aaa2f506f96368a442
utc_time_created: '2022-09-14 17:50:16.578005'


We can also view the dependencies that will be required to serve the model

In [11]:
!cat mlruns/0/$(ls mlruns/0 | head -1)/artifacts/model/conda.yaml

channels:
- conda-forge
dependencies:
- python=3.8.13
- pip<=22.2.2
- pip:
  - mlflow
  - cloudpickle==2.2.0
  - psutil==5.9.2
  - scikit-learn==1.1.2
  - typing-extensions==4.3.0
name: mlflow-env


This file stores the details of how the model was stored. With this information (plus the other files in the folder), we are able to load the model back. Seldon’s MLflow server will use this information to serve this model.

### 4. Push artefacts to MinIO

In [7]:
RUN = "232aa25e011b407aabfae6f92e4c7c8e"
BUCKET_NAME = "wine-model-3"

In [33]:
minioClient = Minio("35.185.70.254:9000", "admin@seldon.io", "12341234", secure=False)

In [34]:
minioClient.list_buckets()

[Bucket('deploy-audit'), Bucket('wine-model-1'), Bucket('wine-model-2')]

In [35]:
!ls mlruns/0/{RUN}/artifacts/model

MLmodel          model.pkl        requirements.txt
conda.yaml       python_env.yaml


In [36]:
def upload_local_directory_to_minio(local_path: str, bucket_name: str, folder_name: str):
    assert os.path.isdir(local_path)

    for local_file in glob.glob(local_path + '/**'):
        local_file = local_file.replace(os.sep, "/")
        if not os.path.isfile(local_file):
            upload_local_directory_to_minio(
                local_file, bucket_name)
        else:
            remote_path = os.path.join(
                local_file[1 + len(local_path):])
            remote_path = remote_path.replace(
                os.sep, "/")
            remote_path = folder_name + "/" + remote_path
            minioClient.fput_object(bucket_name, remote_path, local_file)

In [37]:
upload_local_directory_to_minio(f"mlruns/0/{RUN}/artifacts/model", f"{BUCKET_NAME}", "default")

### 5. Deploy via the SDK

Create a function to authenticate against the cluster

In [1]:
from seldon_deploy_sdk import Configuration, PredictApi, ApiClient, SeldonDeploymentsApi, ModelMetadataServiceApi, DriftDetectorApi, BatchJobsApi, BatchJobDefinition
from seldon_deploy_sdk.auth import OIDCAuthenticator
from seldon_deploy_sdk.rest import ApiException

SD_IP = "35.190.191.28"
config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.username = "admin@seldon.io"
config.password = "12341234"
config.auth_method = "password_grant"

auth = OIDCAuthenticator(config)
config.id_token = auth.authenticate()

api_client = ApiClient(configuration=config, authenticator=auth)

#### Deploy with the v2 protocol
In order to use the v2 protocol, it is best to use conda pack to locally save the conda environment to a tar file.  The initialiser can then use this to install required dependencies into the container.

To use the built-in MLflow server the following pre-requisites need to be met:

* Your MLmodel artifact folder needs to be accessible remotely (e.g. in minio).

* Your model needs to be compatible with the python_function flavour.

* Your MLproject environment needs to be specified using Conda.

In [42]:
!ls mlruns/0 | sed -n 1p

232aa25e011b407aabfae6f92e4c7c8e


In [43]:
!conda pack -o mlruns/0/$(ls mlruns/0 | sed -n 3p)/artifacts/model/environment.tar.gz -f

CondaPackError: 
Files managed by conda were found to have been deleted/overwritten in the
following packages:

- setuptools 63.4.1:
    lib/python3.8/site-packages/pkg_resources/_vendor/__pycache__/zipp.cpython-38.pyc
    lib/python3.8/site-packages/pkg_resources/_vendor/importlib_resources/__init__.py
    lib/python3.8/site-packages/pkg_resources/_vendor/importlib_resources/__pycache__/__init__.cpython-38.pyc
    + 183 others

This is usually due to `pip` uninstalling or clobbering conda managed files,
resulting in an inconsistent environment. Please check your environment for
conda/pip conflicts using `conda list`, and fix the environment by ensuring
only one version of each package is installed (conda preferred).


Push the environment.tar.gz file to MinIO

In [24]:
minioClient.fput_object(f"{BUCKET_NAME}", "default/environment.tar.gz", f"mlruns/0/{RUN}/artifacts/model/environment.tar.gz")

It should be enough to simply specify the v2 protocol.  You no longer need to adjust the liveness and readiness probes.  Note that this deployment can also be done via the UI in Seldon Deploy.

In [2]:
DEPLOYMENT_NAME = f"mlflow-sdk-wine"
NAMESPACE = "seldon"
BUCKET_NAME = "wine-model-3"
MODEL_LOCATION = f"minio:{BUCKET_NAME}/default"
SECRET_NAME = "minio-bucket"


mldeployment = {
  "apiVersion": "machinelearning.seldon.io/v1alpha2",
  "kind": "SeldonDeployment",
  "metadata": {
    "name": f"{DEPLOYMENT_NAME}",
    "namespace": NAMESPACE
  },
  "spec": {
    "protocol": "v2",
    "name": f"{DEPLOYMENT_NAME}",
    "predictors": [
      {
        "graph": {
          "children": [],
          "implementation": "MLFLOW_SERVER",
          "modelUri": f"{MODEL_LOCATION}",
          "envSecretRefName": f"{SECRET_NAME}",
          "name": f"{DEPLOYMENT_NAME}-container",
          "logger": {
                        "mode": "all"
                    }
        },
        "name": "default",
        "replicas": 1
      }
    ]
  }
}

In [3]:
deployment_api = SeldonDeploymentsApi(api_client)
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

{'api_version': None,
 'kind': None,
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': '2022-11-15T18:55:56Z',
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': None,
              'managed_fields': [{'api_version': 'machinelearning.seldon.io/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'.': {},
                                                           'f:name': {},
                                                           'f:predictors': {},
                                                           'f:protocol': {}}},
                                  'manager': 'deployserver',
                                  'operation': 'Update',
                                  'time': '202

### 6. Run an inference request

In [11]:
inference_request = {
    "parameters": {
        "content_type": "pd"
    },
    "inputs": [
        {
          "name": "fixed acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [7.4],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "volatile acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.7000],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "citric acidity",
          "shape": [1],
          "datatype": "FP32",
          "data": [0],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "residual sugar",
          "shape": [1],
          "datatype": "FP32",
          "data": [1.9],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "chlorides",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.076],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "free sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [11],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "total sulfur dioxide",
          "shape": [1],
          "datatype": "FP32",
          "data": [34],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "density",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.9978],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "pH",
          "shape": [1],
          "datatype": "FP32",
          "data": [3.51],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "sulphates",
          "shape": [1],
          "datatype": "FP32",
          "data": [0.56],
          "parameters": {
              "content_type": "np"
          }
        },
        {
          "name": "alcohol",
          "shape": [1],
          "datatype": "FP32",
          "data": [9.4],
          "parameters": {
              "content_type": "np"
          }
        },
    ]
}

In [14]:
import pprint
# create an instance of the API class
predict_api = PredictApi(api_client)


try:
    api_response = predict_api.predict_seldon_deployment(DEPLOYMENT_NAME, NAMESPACE, inference_request)
    # pprint.pprint(api_response)
    print(f"{api_response}")
except ApiException as e:
    print("Exception when calling PredictApi --> predict_seldon_deployment: %s\n" % e)

{'id': '2cd530da-7d41-4a5a-a1df-287281b8c760', 'model_name': 'mlflow-sdk-wine-container', 'model_version': 'v1', 'outputs': [{'data': [5.336543269834809], 'datatype': 'FP64', 'name': 'output-1', 'parameters': None, 'shape': [1]}], 'parameters': {'content_type': None, 'headers': None}}


## Explainer

### Load the model from Storage

In [33]:

minioClient.fget_object('wine-model-2', 'default/model.pkl', './tmp/model.pkl')


In [34]:
import pickle
filename = "./tmp/model.pkl"
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


### Setup Data Set to Train Explainer

In [35]:
from sklearn.model_selection import train_test_split

csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    print(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


In [52]:
X_train_np = train_x.to_numpy()
X_test_np = test_x.to_numpy()

In [46]:
loaded_model.predict(train_x)

array([5.23428491, 5.31135965, 6.12468065, ..., 6.41227667, 4.79863128,
       5.5133657 ])

In [39]:
from alibi.explainers import AnchorTabular

In [40]:
columns = ["fixed acidity", "volatile acidity", "citric acidity", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]

In [49]:
predict_fn = lambda x: loaded_model.predict(x)
explainer = AnchorTabular(predict_fn, columns)


In [50]:
explainer.fit(X_train_np, disc_perc=(25, 50, 75))

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None, 'disc_perc': (25, 50, 75)},
  'version': '0.7.0'}
)

In [54]:
explanation = explainer.explain(X_test_np[3], threshold=0.9)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: 9.50 < alcohol <= 10.20 AND 0.39 < volatile acidity <= 0.51 AND 14.00 < free sulfur dioxide <= 21.00 AND 38.00 < total sulfur dioxide <= 63.00 AND 2.20 < residual sugar <= 2.60 AND 1.00 < density <= 1.00 AND 0.62 < sulphates <= 0.73 AND pH > 3.40 AND fixed acidity <= 7.10 AND 0.08 < chlorides <= 0.09 AND 0.09 < citric acidity <= 0.26
Precision: 0.00
Coverage: 0.00


In [55]:
explainer.save("./tmp/explainer")

In [57]:
upload_local_directory_to_minio(f"tmp/explainer", f"wine-model-2", "explainer")